In [ ]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry

from api_key import APIKEY

API_KEY = APIKEY

In [ ]:
CH4 = ['mp-680722'	, 'mp-31107'	, 'mp-1095378',	 'mp-23569'	, 'mp-30003',	 'mp-29794',	 'mp-1190864'	, 'mp-23084'	, 'mp-15511'	, 'mp-7128',	 'mp-28855'	 ,'mp-561561'	, 'mp-1079364'	, 'mp-558122',	 'mp-542115'	, 'mp-31232'	, 'mp-755116'	, 'mp-1091363'	, 'mp-558910'	, 'mp-14037'	, 'mp-29698',	 'mp-29274'	, 'mp-1095283'	, 'mp-20805'	, 'mp-1113861'	, 'mp-16834',	 'mp-778924',	 'mp-556120', 'mp-20727'	, 'mp-14368'	, 'mp-1819',	 'mp-22467'	, 'mp-753747'	, 'mp-15750',	 'mp-554435'	, 'mp-1105700'	, 'mp-1217378',	 'mp-680219'	, 'mp-17637',	 'mp-555259'	, 'mp-758233'	, 'mp-759342'	, 'mp-23446'	, 'mp-779156'	, 'mp-17677',	 'mp-685353',	 'mp-781788'	, 'mp-230'	, 'mp-31069'	, 'mp-1111368'	, 'mp-759307'	, 'mp-624234']
CHOOH = ['mp-20015'	, 'mp-14367',	 'mp-7094'	, 'mp-541114'	, 'mp-28299'	, 'mp-562338'	, 'mp-760188'	, 'mp-554944'	, 'mp-10798']
CO = ['mp-3414'	, 'mp-542001'	, 'mp-29193'	, 'mp-28845'	, 'mp-561423',	 'mp-14276'	, 'mp-29590',	 'mp-556442'	, 'mp-27980',	 'mp-768936']